In [4]:
import dash
from dash import dcc, html, Input, Output, dash_table
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os

# Initialize the Dash app
app = dash.Dash(__name__, title="IT School Online – Business Operations Overview")
server = app.server

# ======================
# Custom Color Palette
# ======================
color_palette = {
    'blue': '#1E90FF',        # Dodger Blue
    'light_blue': '#87CEFA',   # Light Sky Blue
    'dark_blue': '#00008B',    # Dark Blue
    'purple': '#9370DB',       # Medium Purple (used for KPI text)
    'light_purple': '#D8BFD8', # Thistle
    'turquoise': '#40E0D0',    # Turquoise
    'dark_turquoise': '#00CED1', # Dark Turquoise
    'white': '#FFFFFF',
    'gray': '#F5F5F5',         # Used for KPI card background
    'card_gray': '#E0E0E0'     # Slightly darker gray for better contrast
}

# ======================
# Data Loading Functions
# ======================

def calculate_kpis(deals_path="Deals (Final).xlsx", contacts_path="Contacts (Final).xlsx"):
    """Calculate KPI metrics"""
    kpis = {
        "total_deals": "N/A",
        "closure_time": "N/A",
        "conversion_rate": "N/A",
        "total_revenue_for_successful_deals": "N/A",
        "unique_contacts": "N/A"
    }

    try:
        # Load deals data
        deals = pd.read_excel(deals_path, dtype={'Id': str, 'Contact Name': str})
        if deals.empty:
            return kpis

        # Total Deals
        total_deals = len(deals)
        kpis["total_deals"] = f"{total_deals:,}"

        # Average Deal Closure Time
        if "Closing Date" in deals.columns and "Created Time" in deals.columns:
            closed_deals = deals[deals["Closing Date"].notnull()]
            if not closed_deals.empty:
                closure_times = (closed_deals["Closing Date"] - closed_deals["Created Time"]).dt.days
                if not closure_times.empty:
                    avg_closure = closure_times.mean()
                    kpis["closure_time"] = f"{avg_closure:.1f} Days"

        # Conversion Rate
        if "Stage" in deals.columns:
            successful_deals = deals[deals["Stage"] == "Payment Done"]
            if not successful_deals.empty and total_deals > 0:
                conversion_rate = (len(successful_deals) / total_deals) * 100
                kpis["conversion_rate"] = f"{conversion_rate:.1f}%"

        # Total Revenue
        if "Stage" in deals.columns and "Offer Total Amount" in deals.columns:
            if not successful_deals.empty:
                successful_deals_with_amount = successful_deals[successful_deals["Offer Total Amount"].notnull()]
                if not successful_deals_with_amount.empty:
                    total_revenue = successful_deals_with_amount["Offer Total Amount"].sum()
                    kpis["total_revenue_for_successful_deals"] = f"{total_revenue:,.0f} EURO"

        # Unique Contacts
        contacts = pd.read_excel(contacts_path, dtype={'Id': str})
        if not contacts.empty:
            unique_contacts = len(contacts.drop_duplicates(subset=["Id"]))
            kpis["unique_contacts"] = f"{unique_contacts:,}"

    except Exception as e:
        print(f"Error calculating KPIs: {e}")

    return kpis

def load_deals_data():
    """Load and clean deals data"""
    deals = pd.read_excel('Deals (Final).xlsx', dtype={'Id': str, 'Contact Name': str})
    deals['Stage'] = deals['Stage'].fillna('Unknown')
    deals['Offer Total Amount'] = deals['Offer Total Amount'].fillna(0)
    return deals.dropna(subset=['Id', 'Created Time'])

def load_calls_data():
    """Load and clean calls data"""
    calls = pd.read_excel('Calls (Final).xlsx', dtype={'Id': str, "CONTACTID": str})
    return calls.dropna(subset=['Call Start Time'])

def load_geo_data():
    """Load geographic data"""
    deals = pd.read_excel("Deals (Final).xlsx", dtype={'Id': str, 'Contact Name': str})
    coords_df = pd.read_csv("city_coords.csv")
    deals = pd.merge(deals, coords_df, on="City", how="left")
    deals = deals.dropna(subset=["Latitude", "Longitude", "Level of Deutsch"])
    deals = deals[deals["Level of Deutsch"].str.lower() != 'not defined']
    return deals

# ======================
# Dashboard Layout
# ======================

app.layout = html.Div([
    # Title
    html.H1("IT School Online – Business Operations Overview", 
            style={'textAlign': 'center', 'margin': '20px 0'}),
    
    # First Row - KPI Cards (5 cards)
    html.Div(id='kpi-cards', className='row', style={
        'margin': '20px 0',
        'display': 'flex',
        'justifyContent': 'space-between'
    }),
    
    # Second Row - Calls and Deals per Product, Sales Managers Performance
    html.Div([
        # Calls and Deals per Product
        html.Div([
            html.H3("Calls and Deals per Product", style={'textAlign': 'center'}),
            html.Div([
                html.Label("Select Product:"),
                dcc.Dropdown(
                    id='product-filter',
                    options=[{'label': p, 'value': p} for p in ['All Products'] + 
                            sorted(load_deals_data()['Product'].dropna().unique().tolist())],
                    value='All Products',
                    clearable=False
                )
            ], style={'width': '80%', 'margin': '20px auto'}),
            dcc.Tabs([
                dcc.Tab(label='Monthly Trends', children=[
                    dcc.Graph(id='monthly-trends')
                ]),
                dcc.Tab(label='Detailed Data', children=[
                    dash_table.DataTable(
                        id='data-table',
                        page_size=20,
                        style_table={'overflowX': 'auto'}
                    )
                ])
            ])
        ], className='six columns', style={
            'border': '1px solid #ddd', 
            'padding': '15px', 
            'borderRadius': '5px',
            'backgroundColor': color_palette['gray'],
            'margin': '10px',
            'width': '48%'  # Adjusted for better fit
        }),
        
        # Sales Managers Performance
        html.Div([
            html.H3("Sales Managers Performance", style={'textAlign': 'center'}),
            dcc.Dropdown(
                id='metric-selector',
                options=[
                    {'label': 'Total Deals', 'value': 'Total Deals'},
                    {'label': 'Successful Deals', 'value': 'Successful Deals'},
                    {'label': 'Conversion Rate (%)', 'value': 'Conversion Rate'},
                    {'label': 'Total Sales Amount', 'value': 'Offer Total Amount'}
                ],
                value='Total Deals',
                clearable=False,
                style={'width': '80%', 'margin': '20px auto'}
            ),
            dcc.Graph(id='bar-chart')
        ], className='six columns', style={
            'border': '1px solid #ddd', 
            'padding': '15px', 
            'borderRadius': '5px',
            'backgroundColor': color_palette['gray'],
            'margin': '10px',
            'width': '48%'  # Adjusted for better fit
        })
    ], className='row', style={
        'margin': '20px 0',
        'display': 'flex',
        'justifyContent': 'space-between'
    }),
    
    # Third Row - Sales Funnel Analysis, Geographic Analysis
    html.Div([
        # Sales Funnel Analysis
        html.Div([
            html.H3("Sales Funnel Analysis", style={'textAlign': 'center'}),
            html.Div([
                html.Label("Source:"),
                dcc.Dropdown(
                    id='source-filter',
                    options=[{'label': 'All', 'value': 'All'}] + 
                            [{'label': s, 'value': s} for s in load_deals_data()['Source'].dropna().unique()],
                    value='All',
                    clearable=False,
                    style={'marginBottom': '10px'}
                ),
                html.Label("City:"),
                dcc.Dropdown(
                    id='city-filter',
                    options=[{'label': 'All', 'value': 'All'}] + 
                            [{'label': c, 'value': c} for c in load_deals_data()['City'].dropna().unique()],
                    value='All',
                    clearable=False,
                    style={'marginBottom': '10px'}
                ),
                html.Label("Product:"),
                dcc.Dropdown(
                    id='product-funnel-filter',
                    options=[{'label': 'All', 'value': 'All'}] + 
                            [{'label': p, 'value': p} for p in load_deals_data()['Product'].dropna().unique()],
                    value='All',
                    clearable=False
                )
            ], style={'width': '80%', 'margin': '0 auto'}),
            dcc.Graph(id='funnel-chart')
        ], className='six columns', style={
            'border': '1px solid #ddd', 
            'padding': '15px', 
            'borderRadius': '5px',
            'backgroundColor': color_palette['gray'],
            'margin': '10px',
            'width': '48%'  # Adjusted for better fit
        }),
        
        # Geographic Analysis
        html.Div([
            html.H3("Geographic Analysis", style={'textAlign': 'center'}),
            dcc.Graph(id='geo-map')
        ], className='six columns', style={
            'border': '1px solid #ddd', 
            'padding': '15px', 
            'borderRadius': '5px',
            'backgroundColor': color_palette['gray'],
            'margin': '10px',
            'width': '48%'  # Adjusted for better fit
        })
    ], className='row', style={
        'margin': '20px 0',
        'display': 'flex',
        'justifyContent': 'space-between'
    })
])

# ======================
# Callbacks
# ======================

@app.callback(
    Output('kpi-cards', 'children'),
    Input('kpi-cards', 'id')
)
def update_kpi_cards(_):
    kpis = calculate_kpis()
    
    # Define KPI cards with gray background and purple text
    kpi_cards = [
        {'title': 'Total Deals', 'value': kpis['total_deals']},
        {'title': 'Avg Closure Time', 'value': kpis['closure_time']},
        {'title': 'Conversion Rate', 'value': kpis['conversion_rate']},
        {'title': 'Total Revenue', 'value': kpis['total_revenue_for_successful_deals']},
        {'title': 'Unique Contacts', 'value': kpis['unique_contacts']}
    ]
    
    return [
        html.Div(
            html.Div([
                html.H3(card['title'], style={
                    'textAlign': 'center', 
                    'color': color_palette['purple'], 
                    'marginBottom': '10px'
                }),
                html.H2(card['value'], style={
                    'textAlign': 'center', 
                    'fontWeight': 'bold',
                    'color': color_palette['purple']
                })
            ], style={
                'padding': '15px',
                'borderRadius': '5px',
                'backgroundColor': color_palette['card_gray'],
                'boxShadow': '0 4px 8px 0 rgba(0,0,0,0.2)',
                'transition': '0.3s',
                'height': '100%',
                'display': 'flex',
                'flexDirection': 'column',
                'justifyContent': 'center'
            }),
            className='two columns',
            style={
                'margin': '0 5px',
                'flex': '1',
                'minWidth': '0'
            }
        ) for card in kpi_cards
    ]

@app.callback(
    [Output('monthly-trends', 'figure'),
     Output('data-table', 'columns'),
     Output('data-table', 'data')],
    [Input('product-filter', 'value')]
)
def update_calls_deals(selected_product):
    deals_data = load_deals_data()
    calls_data = load_calls_data()
    
    # Filter data
    deals_filtered = deals_data.copy()
    if selected_product != 'All Products':
        deals_filtered = deals_filtered[deals_filtered['Product'] == selected_product]
    
    calls_filtered = calls_data.copy()
    if selected_product != 'All Products':
        calls_filtered = calls_filtered[calls_filtered['CONTACTID'].isin(deals_filtered['Contact Name'])]
    
    # Process data
    calls_monthly = (
        calls_filtered.assign(Month=calls_filtered['Call Start Time'].dt.to_period('M'))
        .groupby('Month')
        .size()
        .reset_index(name='Calls')
    )
    
    created_monthly = (
        deals_filtered.assign(Month=deals_filtered['Created Time'].dt.to_period('M'))
        .groupby('Month')
        .size()
        .reset_index(name='Created Deals')
    )
    
    successful_monthly = (
        deals_filtered[deals_filtered['Stage'].str.lower() == 'payment done']
        .assign(Month=deals_filtered['Closing Date'].dt.to_period('M'))
        .groupby('Month')
        .size()
        .reset_index(name='Successful Deals')
    )
    
    # Merge data
    df = (
        pd.merge(created_monthly, successful_monthly, on='Month', how='outer')
        .merge(calls_monthly, on='Month', how='outer')
        .fillna(0)
    )
    df['Month'] = df['Month'].astype(str)
    df['Success Rate'] = (df['Successful Deals'] / df['Created Deals'] * 100).round(1)
    
    # Create figure with new color palette
    fig = px.line(
        df,
        x='Month',
        y=['Created Deals', 'Successful Deals', 'Calls'],
        title=f'Calls and Deals Performance ({selected_product})',
        labels={'value': 'Count', 'variable': 'Metric'},
        markers=True,
        color_discrete_sequence=[
            color_palette['blue'],
            color_palette['purple'],
            color_palette['turquoise']
        ]
    )
    
    fig.update_layout(
        plot_bgcolor=color_palette['white'],
        paper_bgcolor=color_palette['white'],
        xaxis=dict(showline=True, linecolor='lightgray', showgrid=False),
        yaxis=dict(showline=True, linecolor='lightgray', showgrid=False),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        hovermode='x unified'
    )
    
    # Prepare table data
    table_columns = [{'name': col, 'id': col} for col in df.columns]
    table_data = df.to_dict('records')
    
    return fig, table_columns, table_data

@app.callback(
    Output('bar-chart', 'figure'),
    [Input('metric-selector', 'value')]
)
def update_sales_managers(selected_metric):
    deals = load_deals_data()
    
    # Compute metrics
    owner_deals = deals.groupby('Deal Owner Name')['Id'].count().reset_index()
    owner_deals.columns = ['Deal Owner Name', 'Total Deals']

    successful_deals = deals[deals['Stage'] == 'Payment Done']
    successful_owner_deals = successful_deals.groupby('Deal Owner Name')['Id'].count().reset_index()
    successful_owner_deals.columns = ['Deal Owner Name', 'Successful Deals']

    total_sales = deals.groupby('Deal Owner Name')['Offer Total Amount'].sum().reset_index()

    # Merge metrics
    owner_performance = pd.merge(owner_deals, successful_owner_deals, on='Deal Owner Name', how='left')
    owner_performance = pd.merge(owner_performance, total_sales, on='Deal Owner Name', how='left')
    owner_performance['Successful Deals'] = owner_performance['Successful Deals'].fillna(0)
    owner_performance['Offer Total Amount'] = owner_performance['Offer Total Amount'].fillna(0)
    owner_performance['Conversion Rate'] = (
        owner_performance['Successful Deals'] / owner_performance['Total Deals']
    ) * 100

    # Sort and create figure with new color palette
    top_df = owner_performance.sort_values(by=selected_metric, ascending=False).head(10)
    
    color_sequence = [
        color_palette['blue'],
        color_palette['light_blue'],
        color_palette['purple'],
        color_palette['light_purple'],
        color_palette['turquoise'],
        color_palette['dark_turquoise'],
        '#A0CBE8',
        '#6A4F8A',
        '#5FA2CE',
        '#AEC7E8'
    ]
    
    fig = px.bar(
        top_df,
        x=selected_metric,
        y='Deal Owner Name',
        orientation='h',
        text=selected_metric,
        color='Deal Owner Name',
        color_discrete_sequence=color_sequence,
        title=f"Top 10 Deal Owners by {selected_metric}"
    )
    
    fig.update_layout(
        xaxis_title=selected_metric,
        yaxis_title="Deal Owner",
        height=500,
        showlegend=False,
        plot_bgcolor=color_palette['white'],
        paper_bgcolor=color_palette['white']
    )
    
    return fig

@app.callback(
    Output('funnel-chart', 'figure'),
    [Input('source-filter', 'value'),
     Input('city-filter', 'value'),
     Input('product-funnel-filter', 'value')]
)
def update_funnel(source, city, product):
    deals = load_deals_data()
    
    # Apply filters
    filtered_df = deals.copy()
    if source != 'All':
        filtered_df = filtered_df[filtered_df['Source'] == source]
    if city != 'All':
        filtered_df = filtered_df[filtered_df['City'] == city]
    if product != 'All':
        filtered_df = filtered_df[filtered_df['Product'] == product]
    
    # Get stage order by frequency
    stage_order = filtered_df['Stage'].value_counts().index.tolist()
    
    # Prepare funnel data
    funnel_data = (
        filtered_df.groupby('Stage')
        .agg(
            Count=('Id', 'count'),
            Avg_SLA=('SLA in Seconds', lambda x: round(x.mean()/3600, 1) if x.notna().any() else 0),
            Conversion_Rate=('Stage', lambda x: (x == stage_order[-1]).sum()/len(x) if len(x) > 0 else 0),
            Avg_Amount=('Initial Amount Paid', lambda x: round(x.mean(), 2) if x.notna().any() else 0)
        )
        .reindex(stage_order)
        .reset_index()
    )
    
    funnel_data['Stage_Conversion'] = funnel_data['Count'].pct_change(fill_method=None).fillna(0).abs()
    
    # Create visualization with new color palette
    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "funnel"}, {"type": "bar"}]])
    
    # Main funnel plot with new colors
    fig.add_trace(go.Funnel(
        y=funnel_data['Stage'],
        x=funnel_data['Count'],
        textposition="inside",
        textinfo="value+percent initial",
        texttemplate="<b>%{y}</b><br>%{x} deals<br>%{percentInitial:.1%} of leads",
        hoverinfo="text+y+x",
        hovertext=[
            f"<b>{row['Stage']}</b><br>"
            f"Success rate: {row['Conversion_Rate']:.1%}<br>"
            f"Avg payment: €{row['Avg_Amount']:,.2f}<br>"
            f"SLA: {row['Avg_SLA']} hrs"
            for _, row in funnel_data.iterrows()
        ],
        marker={
            "color": [
                color_palette['blue'],
                color_palette['purple'],
                color_palette['turquoise'],
                color_palette['dark_blue'],
                color_palette['dark_turquoise']
            ],
            "line": {"width": 1, "color": "white"}
        },
        connector={"line": {"color": "#7F7F7F", "width": 1}}
    ), row=1, col=1)
    
    # Conversion bar chart with new color
    fig.add_trace(go.Bar(
        x=funnel_data['Stage'],
        y=funnel_data['Stage_Conversion'],
        name='Stage conversion',
        marker_color=color_palette['light_purple'],
        text=[f"{x:.1%}" for x in funnel_data['Stage_Conversion']],
        textposition='auto',
        hoverinfo="text+y",
        hovertext=[
            f"Conversion {stage_order[i]} → {stage_order[i+1]}: {row['Stage_Conversion']:.1%}"
            for i, (_, row) in enumerate(funnel_data.iterrows()) if i < len(stage_order)-1
        ]
    ), row=1, col=2)
    
    # Layout configuration
    title_text = "Sales Funnel Analysis"
    filters = []
    if source != 'All':
        filters.append(f"Source: {source}")
    if city != 'All':
        filters.append(f"City: {city}")
    if product != 'All':
        filters.append(f"Product: {product}")
    
    if filters:
        title_text += "<br><sup>" + " | ".join(filters) + "</sup>"
    
    fig.update_layout(
        title={
            'text': title_text,
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top'
        },
        margin={"l": 50, "r": 50, "t": 100, "b": 50},
        plot_bgcolor=color_palette['white'],
        paper_bgcolor=color_palette['white'],
        font=dict(family="Arial", size=12),
        hoverlabel=dict(bgcolor="white", font_size=12, font_family="Arial"),
        showlegend=False,
        height=500
    )
    
    return fig

@app.callback(
    Output('geo-map', 'figure'),
    Input('geo-map', 'id')
)
def update_geo_map(_):
    deals = load_geo_data()
    
    # Group by city
    grouped = deals.groupby("City").agg({
        "Id": "count",
        "Offer Total Amount": "mean",
        "Level of Deutsch": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "Source": lambda x: x.mode().iloc[0] if not x.mode().empty else None,
        "Latitude": "first",
        "Longitude": "first"
    }).rename(columns={
        "Id": "Deals Count",
        "Offer Total Amount": "Average LTV",
        "Level of Deutsch": "Most Common Deutsch Level",
        "Source": "Most Common Source"
    }).reset_index()

    # Create map with new color scale
    fig = px.scatter_map(
        grouped,
        lat="Latitude",
        lon="Longitude",
        size="Deals Count",
        color="Average LTV",
        hover_name="City",
        hover_data={
            "Deals Count": True,
            "Average LTV": ":.2f",
            "Most Common Deutsch Level": True,
            "Most Common Source": True
        },
        color_continuous_scale=[color_palette['light_blue'], color_palette['blue'], color_palette['purple']],
        size_max=30,
        zoom=3,
        height=600
    )
    
    fig.update_layout(
        mapbox_style="open-street-map",
        margin={"r": 0, "t": 0, "l": 0, "b": 0},
        plot_bgcolor=color_palette['white'],
        paper_bgcolor=color_palette['white']
    )
    
    return fig

# Run the app
if __name__ == '__main__':
    app.run(debug=True, port=8059)